User info I extracted:  
(data dictionary: https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet)
* 'id'
* 'name'
* 'screen_name'
* 'location'
* 'created_at'
* 'followers_count'
* 'friends_count'      (
* 'statuses_count'     (The total number of tweets that the account has posted.)
* 'favourites_count'   (The total number of tweets that the account has liked.)


# Set up your PostgreSQL server
1.    Download and install Postgres.app from https://postgresapp.com/
2.   Open Postgres.app, toggle "start".
3.   Execute the code below to extract the records. Ensure that the files "corona-out-3" and "corona-out-3" are placed in the same working directory.

In [7]:
import psycopg2

In [8]:
# prompting username input from the user
username = input("Please enter your computer's username: ")

In [9]:
# printing the username used
username

'varshiniyanamandra'

In [10]:
# Connect to the default "postgres" database to perform operations on databases
conn = psycopg2.connect(
    dbname="postgres",
    user=username,  # Only replace with your system user name
    password="",
    host="localhost",
    port="5432")

# Open a cursor to perform database operations
cur = conn.cursor()

# Check if the "twitter" database exists
cur.execute("SELECT 1 FROM pg_database WHERE datname = 'twitter';")
exists = cur.fetchone()

# Close the cursor and connection
cur.close()
conn.close()

if exists:
    # Create a new connection with autocommit enabled
    conn = psycopg2.connect(
        dbname="postgres",
        user=username,  # Only replace with your system user name
        password="",
        host="localhost",
        port="5432")
    conn.autocommit = True

    # Open a cursor to perform database operations
    cur = conn.cursor()

    # Drop and recreate the "twitter" database
    cur.execute("DROP DATABASE twitter;")
    cur.execute("CREATE DATABASE twitter;")

    # Close the cursor and connection
    cur.close()
    conn.close()
else:
    # Create a new connection with autocommit enabled
    conn = psycopg2.connect(
        dbname="postgres",
        user=username,  # Only replace with your system user name
        password="",
        host="localhost",
        port="5432")
    conn.autocommit = True

    # Open a cursor to perform database operations
    cur = conn.cursor()

    # create a new database named 'twitter'
    cur.execute("CREATE DATABASE twitter;")

    # Close the cursor and connection
    cur.close()
    conn.close()

# Connect to the newly created "twitter" database
conn = psycopg2.connect(
    dbname="twitter",
    user=username,  # Only replace with your system user name
    password="",
    host="localhost",
    port="5432")

# Open a cursor to perform further database operations
cur = conn.cursor()

print("PostgreSQL server information")
print(conn.get_dsn_parameters(), "\n")


PostgreSQL server information
{'user': 'varshiniyanamandra', 'passfile': '/Users/varshiniyanamandra/.pgpass', 'channel_binding': 'prefer', 'dbname': 'twitter', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [11]:
# Create the User table if it doesn't already exist
cur.execute('''
    CREATE TABLE IF NOT EXISTS TwitterUser (
        id BIGINT PRIMARY KEY,
        name TEXT,
        screen_name TEXT,
        location TEXT,
        created_at TIMESTAMP,
        followers_count INTEGER,
        friends_count INTEGER,
        statuses_count INTEGER,
        favourites_count INTEGER
    );
''')
conn.commit()  

In [12]:
def view_users(conn):
        try:
            cur.execute("SELECT * FROM TwitterUser;")
            users = cur.fetchall()
            if len(users)==0:
                print("No data")
            print("Number of user IDs:",len(users))
        except Exception as e:
            print(f"An error occurred: {e}")
            conn.rollback()

# Use this function to view the data in the TwitterUser table
view_users(conn)

No data
Number of user IDs: 0


In [13]:
import psycopg2
import json

def insert_user(cur, user_info):
        try:
            # Check if the user already exists in the database
            cur.execute("SELECT id FROM TwitterUser WHERE id = %s;", (user_info["id"],))
            if cur.fetchone() is None:
                # If the user does not exist, insert their information into the TwitterUser table
                cur.execute("""
                    INSERT INTO TwitterUser (id, name, screen_name, location, created_at, followers_count, friends_count, statuses_count, favourites_count)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
                """, (user_info["id"], user_info["name"], user_info["screen_name"], user_info["location"], user_info["created_at"], user_info["followers_count"], user_info["friends_count"], user_info["statuses_count"], user_info["favourites_count"]))
                conn.commit()
        except Exception as e:
            print(f"An error occurred: {e}")
            conn.rollback()

def insert_records(file):
    with open(file, "r") as f1:
        for line in f1:
            try:
                tweet = json.loads(line)

                # Extract the user information from the tweet
                user_info = tweet["user"]
                # Insert the user information into the User table
                insert_user(cur, user_info)

                # Check if the tweet is a retweet
                if "retweeted_status" in tweet:
                    # Extract the original tweet's user information
                    original_user_info = tweet["retweeted_status"]["user"]
                    # Insert the original tweet's user information into the User table
                    insert_user(cur, original_user_info)

            except:
                # if there is an error loading the json of the tweet, skip
                continue


In [14]:
#insert_records("corona-out-2")

In [15]:
#view_users(conn)

In [16]:
# cur.execute("SELECT * FROM TwitterUser;")
# users=cur.fetchall()
# for people in users:
#     print(people)

In [18]:
insert_records("corona-out-3")

In [19]:
view_users(conn)

Number of user IDs: 88068


In [20]:
# # Transform the data into a DataFrame for improved visualization and readability. 

# import pandas as pd
# from sqlalchemy import create_engine

# # Create an SQLAlchemy engine using the psycopg2 connection string
# url = "postgresql+psycopg2://" + username + ":@localhost:5432/twitter"
# # engine = create_engine("postgresql+psycopg2://varshiniyanamandra:@localhost:5432/twitter")
# engine = create_engine(url)

# def get_twitter_users_dataframe(engine):
#     query = "SELECT * FROM TwitterUser;"
#     df = pd.read_sql_query(query, engine)
#     return df

# # Fetch the data and convert it into a DataFrame
# user_df = get_twitter_users_dataframe(engine)


In [21]:
# user_df.head(20)

,id,name,screen_name,location,created_at,followers_count,friends_count,statuses_count,favourites_count
0,804046791348015107,Bi Sex Uau,B_King69,"Acre, Brasil",2016-11-30 19:37:48,89,173,4728,5446
1,2242948745,Thomas Krause,tho1965,None,2013-12-25 09:13:33,173,685,1865,2184
2,908326492718764034,शचीन्द्र पाण्डेय,im_S_pandey,Amethi Uttar Pradesh,2017-09-14 13:48:06,2362,202,48906,30668
3,207809313,BJP,BJP4India,"6-A, Deen Dayal Upadhyay Marg,",2010-10-26 02:19:07,13481667,3,200315,0
4,2929344220,Ralf Schmitz,RusticusArat,🇩4790 Provinz,2014-12-18 10:19:26,778,733,30551,32024
5,1206650133976408064,Büşra Öztaş,schrodingerk42,None,2019-12-16 18:59:53,318,220,405,1974
6,1248123252,minhyuk.,mizhyuklee,None,2013-03-07 07:21:32,1194,538,157680,1241
7,858859031464751104,Monsta X Europe,MonstaXEurope,let the haters bark,2017-05-01 01:42:04,27085,463,38045,32014
8,50993809,🇮🇹Henry Whites♥️,Enrico_Bianchi,None,2009-06-26 09:37:39,1165,1840,42671,22923
9,761787475,Maurizio Gustinicchi,gustinicchi,www.scenarieconomici.it,2012-08-16 15:43:06,13459,7886,85444,185978


In [22]:
cur.close()
conn.close()